In [1]:
file_location = "./data/ks_30_0"
with open(file_location,"r") as input_data_file:
    input_data = input_data_file.read()
    
print(input_data)

30 100000
90000 90001
89750 89751
10001 10002
89500 89501
10252 10254
89250 89251
10503 10506
89000 89001
10754 10758
88750 88751
11005 11010
88500 88501
11256 11262
88250 88251
11507 11514
88000 88001
11758 11766
87750 87751
12009 12018
87500 87501
12260 12270
87250 87251
12511 12522
87000 87001
12762 12774
86750 86751
13013 13026
86500 86501
13264 13278
86250 86251



In [2]:
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])

In [3]:
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count + 1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i - 1, int(parts[0]), int(parts[1])))

## Dynamic Programming

In [4]:
import numpy as np
weights = [0]
values = [0]
for i in range(len(items)):
    weights.append(items[i].weight)
    values.append(items[i].value)

In [5]:
K = np.zeros([capacity + 1, item_count + 1])
for i in range(1,item_count+1):
    for w in range(capacity+1):
        if i==0 or w==0:
            K[w,i]=0
        elif w<weights[i]:
            K[w,i]= K[w,i-1]
        else:
            K[w,i]=np.maximum(K[w-weights[i],i-1]+values[i], K[w,i-1])

In [6]:
ctr = 0
x = [0]*item_count
remove_weight=0
max_val = np.max(K)
for i in range(item_count,0,-1):
    if K[-1 - remove_weight,i-1] != K[-1 - remove_weight,i]:
        max_val = K[-1-remove_weight,i]
        remove_weight += weights[i]
        x[i-1] = 1

x

[0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [7]:
total_weight=0
for i in range(len(x)):
    total_weight += x[i]*values[i+1]
total_weight

99798

## Branch and Bound

In [1]:
import numpy as np
import queue

In [2]:
file_location = "./data/ks_4_0"
with open(file_location,"r") as input_data_file:
    input_data = input_data_file.read()
    
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])

lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count + 1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i - 1, int(parts[0]), int(parts[1])))

In [3]:
values = []
weights = []
for item in items:
    values.append(item.value)
    weights.append(item.weight)
density = [v/w for v,w in zip(values,weights)]

In [39]:
values = [45,48,35]
weights = [5,8,3]
capacity = 10
densities = [v/w for v,w in zip(values,weights)]
item_count = len(values)

In [17]:
idxs = list(range(item_count))
idxs.sort(key=lambda i: values[i]/weights[i], reverse=True)

In [18]:
idxs

[0, 1, 2, 3]

In [4]:
class Node:
    def __init__(self, value, weight, estimate, level, contains):
        self.value = value
        self.weight = weight
        self.estimate = estimate
        self.level = level
        self.contains = contains
        

Depth-First Branch

In [5]:
q = queue.Queue()

In [44]:
value = 0
best = []
ctr = 0

root = Node(value=0, weight=0, estimate=sum(values), level=0, contains=[])
q = queue.Queue()
q.put(root)
while not q.empty():
    n = q.get()
    
    if n.level < item_count:
        left = Node(value=n.value+values[n.level], weight=n.weight+weights[n.level], estimate=n.estimate, level=n.level+1, contains=n.contains[:])
        left.contains.append(1)
    
        if left.weight<=capacity:
            if left.value > value:
                value = left.value
                best = left.contains
            if left.estimate > value:
                q.put(left)
            
    
        right = Node(value=n.value, weight=n.weight, estimate=n.estimate-values[n.level], level=n.level+1, contains=n.contains[:])
        right.contains.append(0)

        if right.weight<=capacity:
            if right.value > value:
                value = right.value
                best = right.contains
            if right.estimate > value:
                q.put(right)
            
print("Value:",value,"\tX:", best)

Value: 80 	X: [1, 0, 1]


Better Relaxation

In [49]:
sorted_args = list(reversed(np.argsort(densities)))
sorted_args

[2, 0, 1]

In [59]:
x_take = [0]*item_count
best = [1,1,0]

for item, take in zip(sorted_args,range(len(best))):
    x_take[item] = best[take]

[1, 0, 1]

In [53]:
optimistic_estimation = 0
w = 0
ctr = 0
while w < capacity:
    optimistic_estimation += values[sorted_args[ctr]]
    w += weights[sorted_args[ctr]]
    ctr+=1

In [62]:
items.sort(key=lambda x: x.value/x.weight, reverse=True)
values = []
weights = []
for item in items:
    values.append(item.value)
    weights.append(item.weight)

In [ ]:
value = 0
best = []
ctr = 0

root = Node(value=0, weight=0, estimate=sum(values), level=0, contains=[])
q = queue.Queue()
q.put(root)
while not q.empty():
    n = q.get()
    
    if n.level < item_count:
        left = Node(value=n.value+values[n.level], weight=n.weight+weights[n.level], estimate=n.estimate, level=n.level+1, contains=n.contains[:])
        left.contains.append(1)
    
        if left.weight<=capacity:
            if left.value > value:
                value = left.value
                best = left.contains
            if left.estimate > value:
                q.put(left)
            
    
        right = Node(value=n.value, weight=n.weight, estimate=n.estimate-values[n.level], level=n.level+1, contains=n.contains[:])
        right.contains.append(0)

        if right.weight<=capacity:
            if right.value > value:
                value = right.value
                best = right.contains
            if right.estimate > value:
                q.put(right)
            
print("Value:",value,"\tX:", best)

Best-First

In [43]:
top_value = 0
best = []
ctr = 0

root = Node(value=0, weight=0, estimate=sum(values), level=0, contains=[])
list_nodes = []
list_nodes.append(root)

for i in range(2):
#while list_nodes != []:
    n = list_nodes.pop()
    
    if n.level < item_count:
        left = Node(value=n.value+values[n.level], weight=n.weight+weights[n.level], estimate=n.estimate, level=n.level+1, contains=n.contains[:])
        left.contains.append(1)
        print("left\tEstimate: {}\tValue:{}".format(left.estimate, left.value))
    
        if left.weight<=capacity:
            if left.value>=top_value:         
                top_value = left.value
                best = left.contains
            if left.estimate>=value:
                list_nodes.append(left)
    
        right = Node(value=n.value, weight=n.weight, estimate=n.estimate-values[n.level], level=n.level+1, contains=n.contains[:])
        right.contains.append(0)
        print("right\tEstimate: {}\tValue:{}".format(right.estimate, right.value))
        if right.weight<=capacity:
            if right.value>=top_value:         
                top_value = right.value
                best = right.contains
            if right.estimate>=value:
                list_nodes.append(right)
        list_nodes.sort(key=lambda x: x.estimate)

    ctr+=1

print("Value:",top_value,"\tX:", best)

left	Estimate: 128	Value:45
right	Estimate: 83	Value:0
left	Estimate: 128	Value:93
right	Estimate: 80	Value:45
Value: 45 	X: [1, 0]


In [18]:
list_nodes.sort(key=lambda x: x.value, reverse=True)

In [34]:
ctr

11

In [14]:
list_nodes[0].value

18

In [ ]:
x = [0]*item_count
current_weight = 0
current_value = 0
estimate = sum(values)
print("Weight: {}\tValue: {}\tEstimate: {}".format(current_weight, current_value, estimate))

In [ ]:
print(values)
print(weights)
print(density)

In [ ]:
sorted_args = list(reversed(np.argsort(density)))

In [ ]:
for i in sorted_args:
    print(weights[i])

In [ ]:
class Node:
    def __init__(self, value, weight, estimate):
        self.value = value
        self.weight = weight
        self.estimate = estimate
        

In [ ]:
list_nodes=[]
root = Node(value=0, weight=0, estimate=sum(weights))
list_nodes.append(n0)

In [ ]:
for node in list_nodes:
    print(node.value)

In [ ]:
import queue

In [ ]:
q = queue.Queue()

In [ ]:
q.put(root)

In [ ]:
q.get()